In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import ast
import uuid

THIS SCRIPT WILL WRITE AN XML FILE THAT YOU CAN PLACE IN YOUR GRASSHOPPER SETTINGS FOLDER TO GET NICE GRADIENTS

In [2]:
#All available colormaps in matplotlib. 
pus = ['viridis', 'plasma', 'inferno', 'magma', 'cividis']
seq1 = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                      'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                      'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
seq2 =  ['binary', 'gist_yarg', 'gist_gray', 'gray', 'bone',
                      'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
                      'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper']
div = ['PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu',
                      'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']
cyclic = ['twilight', 'twilight_shifted', 'hsv']
misc = ['flag', 'prism', 'ocean', 'gist_earth', 'terrain',
                      'gist_stern', 'gnuplot', 'gnuplot2', 'CMRmap',
                      'cubehelix', 'brg', 'gist_rainbow', 'rainbow', 'jet',
                      'turbo', 'nipy_spectral', 'gist_ncar']

names = pus+seq1+seq2+div+cyclic+misc

In [3]:
#Color maps that we actually want to use
pus = ['viridis', 'plasma', 'inferno', 'magma', 'cividis']
seq1 = ['Blues', 'Greens', 'YlGnBu']
seq2 =  ['binary', 'bone',
                      'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
                      'Wistia', 'hot', 'copper']
div = ['PiYG', 'PRGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']
cyclic = []
misc = [ 'ocean', 
                      'turbo', ]

names = pus+seq1+seq2+div+cyclic+misc

In [4]:
allcolors = []
noOfDiv = 2**3
for cmap in names:
    colormaplist = plt.get_cmap(cmap)
    colors = []
    for i in range(int(256/(noOfDiv))+1):
        color = list(colormaplist(i*noOfDiv))
        color = [round(num, 5) for num in color]
        colors.append(color)
    allcolors.append(colors)

In [5]:
str0 = """
<Fragment name="Settings">
  <items count="XXX">
"""
str0 = str0.replace("XXX",str(len(allcolors)))



In [6]:
str1 = """    <item name="preset:INDEX" type_name="gh_string" type_code="10">&lt;Fragment name="gradient"&gt;
  &lt;items count="3"&gt;
    &lt;item name="GripCount" type_name="gh_int32" type_code="3"&gt;GRIPCOUNT&lt;/item&gt;
    &lt;item name="Linear" type_name="gh_bool" type_code="1"&gt;false&lt;/item&gt;
    &lt;item name="Locked" type_name="gh_bool" type_code="1"&gt;false&lt;/item&gt;
  &lt;/items&gt;
  &lt;chunks count="TOTALCOUNT"&gt;
    XXX
  &lt;/chunks&gt;
&lt;/Fragment&gt;</item>"""

In [7]:
str2 ="""&lt;chunk name="Grip" index="COLORINDEX"&gt;
      &lt;items count="4"&gt;
        &lt;item name="Colour" index="0" type_name="gh_drawing_color" type_code="36"&gt;
          &lt;ARGB&gt;255;RRR;GGG;BBB&lt;/ARGB&gt;
        &lt;/item&gt;
        &lt;item name="Colour" index="1" type_name="gh_drawing_color" type_code="36"&gt;
          &lt;ARGB&gt;255;RRR;GGG;BBB&lt;/ARGB&gt;
        &lt;/item&gt;
        &lt;item name="Factor" type_name="gh_double" type_code="6"&gt;LOCATION&lt;/item&gt;
        &lt;item name="Id" type_name="gh_guid" type_code="9"&gt;GUIDGUID&lt;/item&gt;
      &lt;/items&gt;
    &lt;/chunk&gt;
    XXX"""

In [8]:
str3="""  </items>
</Fragment>"""

In [9]:
outStr =""
for i,c in enumerate(allcolors):
    tempString = str1
    tempString = tempString.replace("INDEX",str(i))
    tempString = tempString.replace("TOTALCOUNT",str(len(c)))
    tempString = tempString.replace("GRIPCOUNT",str(len(c)))
    for j,color in enumerate(c):
        tempString2 = str2
        tempString2 = tempString2.replace("RRR",str(int(255*color[0])))
        tempString2 = tempString2.replace("GGG",str(int(255*color[1])))
        tempString2 = tempString2.replace("BBB",str(int(255*color[2])))
        tempString2 = tempString2.replace("LOCATION",str(j/(len(c)-1)))
        tempString2 = tempString2.replace("COLORINDEX",str(j))
        tempString2 = tempString2.replace("GUIDGUID",str(uuid.uuid4()))
        tempString = tempString.replace("XXX",tempString2)
    tempString = tempString.replace("XXX","")
    outStr +=tempString
outStr = str0 + outStr+ str3


In [10]:
f = open("grasshopper_gradients.xml", "w")
f.write(outStr)
f.close()